In [19]:
import zipfile
import requests

url = requests.get("https://www.dt.fee.unicamp.br/~tiago/smsspamcollection/smsspamcollection.zip")
with open("sms.zip", "wb") as f:
  f.write(r.content)

zipfile.ZipFile("sms.zip").extractall()


In [20]:
data = []
with open("SMSSpamCollection.txt", "r") as f:
  for line in f:
    data.append(line.strip().split("\t"))

data

[['ham',
  'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'],
 ['ham', 'Ok lar... Joking wif u oni...'],
 ['spam',
  "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"],
 ['ham', 'U dun say so early hor... U c already then say...'],
 ['ham', "Nah I don't think he goes to usf, he lives around here though"],
 ['spam',
  "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"],
 ['ham',
  'Even my brother is not like to speak with me. They treat me like aids patent.'],
 ['ham',
  "As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune"],
 ['spam',
  'WINNER!! As a valued network customer you have been selected to receivea £

In [21]:
import re 

for d in data:
  d[1] = set(re.findall("[0-9a-z_]+", d[1].lower()))

In [25]:
random.shuffle(data)

train_size = int(len(data) * 0.8)
test_size = len(data) - train_size

train = data[:train_size]
test = data[train_size:]

print(len(train), len(test))

print(train)

4459 1115
[['ham', {'now', 'is', 'k', 'how', 'business', 'your'}], ['ham', {'on', 'very', 'thursday', 'be', 'nice', 'ready', 'ya'}], ['ham', {'love', 'lot', 'my', 'you', 'dear', 'umma', 'vava', 'and', 'life'}], ['ham', {'i', 'didnt', 'you', 'no', 'know', 'let', 'online', 'and', 'she', 'search', 'will'}], ['ham', {'i', 'okay', 'you', 'were', 'thought', 'but', 'the', 'expert'}], ['ham', {'m', 'i', 'done'}], ['ham', {'mind', 'my', 'ni8', 'but', 'dreams', 't', 'never', 'with', 'sweet', 'just', 'someone', 'tolerat', 'lov', 'gud', 'this', 'comfort', 'i', 'truth', 'line', 'me', 'wil', 'a', 'bcs', 'don', 'hurt', 'ur', 'lie', 'and', 'the'}], ['ham', {'w', 'doing', 'project', 'lor', 'frens'}], ['ham', {'do', 'again', 'you', 'beer', 'there', 'now', 'house', 'any', 'whats', 'and', 'your', 'have'}], ['spam', {'urgent', 'won', 'attempt', '50', 'have', 'to', '09071512432', '1000call', 'callcost150ppmmobilesvary', 'this', 'contact', 'csbcm4235wc1n3xx', 'b4', 'is', '7', 'u', 'max', 'the', '300603t', '2

In [30]:
# 사전확률 구하기

n_total = len(train)
n_spam = sum(1 for cls, bow in train if cls == 'spam')
n_ham = sum(1 for cls, bow in train if cls == 'ham')

alpha = 0.5

prior_spam = (n_spam + alpha) / (n_total + 2*alpha)
prior_ham = (n_ham + alpha) / (n_total + 2 * alpha)

print(prior_spam, prior_ham)

# prior_spam = math.log(len(train_spam) / len(train))
# train_ham = [bow for cls, bow in train if cls =='ham']

# prior_spam = math.log(len(train_spam) / len(train))
# prior_ham = math.log(len(train_ham) / len(train))

0.1312780269058296 0.8687219730941704


In [35]:
# 가능도

# spam_words = {}
# ham_words = {}

# for cls, bow in train:
#   for word in bow:
#     if cls == 'spam':
#       if word not in spam_words:
#         spam_words[word] = 0
#       spam_words[word] += 1
#       if cls == 'ham':
#         if word not in ham_words:
#           ham_words[word] = 0
#         ham_words[word] += 1

# spam_words

from collections import Counter

spam_words = Counter(word for cls, bow in train for word in bow if cls == 'spam')
ham_words = Counter(word for cls, bow in train for word in bow if cls == 'ham')

In [37]:
def predict(bow, spam_words, ham_words, n_spams, n_hams, alpha):
  spam_score = prior_spam + sum(math.log((spam_words[word] + alpha) / (n_spams + alpha)) for word in bow)
  ham_score = prior_ham + sum(math.log((ham_words[word] + alpha) / (n_hams + alpha)) for word in bow)

  if spam_score < ham_score:
    return 'ham'
  else:
    return 'spam'

In [39]:
tp, tn, fp, fn = 0, 0, 0, 0

for cls, bow in test:
  pred = predict(bow, spam_words, ham_words, len(train_spam), len(train_ham), 1e-4)
  if cls == 'spam' and pred == 'spam' : tp += 1
  elif cls == 'ham' and pred == 'ham' : tn += 1
  elif cls == 'ham' and pred == 'spam' : fp += 1
  else: fn += 1

print(cls, pred)

ham ham


In [40]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)
print(accuracy, precision, recall, f1)

0.9560538116591928 0.7897435897435897 0.9506172839506173 0.8627450980392156
